In [1]:
import $plugin.$ivy.`org.spire-math::kind-projector:0.9.4`
import $plugin.$ivy.`org.scalamacros:paradise_2.11.11:2.1.0`
import $ivy.`com.thoughtworks.deeplearning::plugins-builtins:2.0.0`
import $ivy.`com.thoughtworks.each::each:3.3.1`
import $ivy.`org.nd4j:nd4j-native-platform:0.8.0`
import $ivy.`org.rauschig:jarchivelib:0.5.0`

import $plugin.$                                          

import $plugin.$                                            

import $ivy.$                                                      

import $ivy.$                                  

import $ivy.$                                    

import $ivy.$                               

In [2]:
import scala.concurrent.ExecutionContext.Implicits.global
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.convolution.Convolution
import org.nd4j.linalg.util.ArrayUtil
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.api.ops.impl.transforms.IsMax
import scalaz.syntax.all._
import scalaz.std.anyVal._
import com.thoughtworks.future._
import com.thoughtworks.raii.shared._
import com.thoughtworks.raii.asynchronous._
import com.thoughtworks.feature.ImplicitApply
import com.thoughtworks.each.Monadic._
import com.thoughtworks.feature.Factory
import com.thoughtworks.deeplearning.DeepLearning
import com.thoughtworks.deeplearning.DeepLearning.Tape
import com.thoughtworks.deeplearning.plugins.INDArrayWeights
import com.thoughtworks.deeplearning.plugins.INDArrayLayers
import com.thoughtworks.deeplearning.plugins.Operators
import com.thoughtworks.deeplearning.plugins.Training
import com.thoughtworks.deeplearning.plugins.Builtins
import com.thoughtworks.deeplearning.plugins.ImplicitsSingleton

import scala.concurrent.ExecutionContext.Implicits.global

import org.nd4j.linalg.api.ndarray.INDArray

import org.nd4j.linalg.convolution.Convolution

import org.nd4j.linalg.util.ArrayUtil

import org.nd4j.linalg.factory.Nd4j

import org.nd4j.linalg.api.ops.impl.transforms.IsMax

import scalaz.syntax.all._

import scalaz.std.anyVal._

import com.thoughtworks.future._

import com.thoughtworks.raii.shared._

import com.thoughtworks.raii.asynchronous._

import com.thoughtworks.feature.ImplicitApply

import com.thoughtworks.each.Monadic._

import com.thoughtworks.feature.Factory

import com.thoughtworks.deeplearning.DeepLearning

import com.thoughtworks.deeplearning.DeepLearning.Tape

import com.thoughtworks.deeplearning.plugins.INDArrayWeights

import com.thoughtworks.deeplearning.plugins.INDArrayLayers

import com.thoughtworks.deeplearning.plugins.Operators

import com.thoughtworks.deeplearning.plugins.Training

import com.thoughtworks.deeplearning.plugins.Builtins

import com.thoughtwo

In [6]:
interp.load(scala.io.Source.fromURL(new java.net.URL("https://gist.githubusercontent.com/TerrorJack/cdd9cc5adc82fc86abf8b4c72cd26e76/raw/1f15523ee4b5a7fcc7e7317ae34ba09be207a62a/CNN.sc")).mkString)

interp.load(scala.io.Source.fromURL(new java.net.URL("https://gist.github.com/Atry/1fb0608c655e3233e68b27ba99515f16/raw/39ba06ee597839d618f2fcfe9526744c60f2f70a/FixedLearningRate.sc")).mkString)

interp.load(scala.io.Source.fromURL(new java.net.URL("https://gist.github.com/Atry/89ee1baa4c161b8ccc1b82cdd9c109fe/raw/d39cec0483a78a514342b16567aebebe0f890d38/Adagrad.sc")).mkString)

In [7]:
val NumberOfFilters = 128

NumberOfFilters: Int = 128

In [8]:
import java.io._
import java.net.URI
import java.nio.file._

trait BranchNet extends CNNs with Builtins with Adagrad {
    val fileHandler = new java.util.logging.FileHandler(s"BranchNet-${management.ManagementFactory.getRuntimeMXBean.getName}-%g.log")
    logger.addHandler(fileHandler)

    final case class WeightFile(weight: INDArrayWeight, relativePath: Path) {
        def backup(parent: Path): Unit = {
            val resolvedPath = parent.resolve(relativePath)
            Files.createDirectories(resolvedPath.getParent)
            val stream = new ObjectOutputStream(new BufferedOutputStream(Files.newOutputStream(resolvedPath, StandardOpenOption.CREATE_NEW)))
            try {
                stream.writeObject(weight.data)
                stream.writeObject(weight.cache)
            } finally {
                stream.close()
            }
        } 
    }

    trait ImplicitsApi extends super[CNNs].ImplicitsApi with super[Builtins].ImplicitsApi
    type Implicits <: ImplicitsApi
    
    import implicits._

    def softmax[Out <: INDArrayLayer](scores: INDArrayLayer)(implicit layerImplicits: ImplicitApply.Aux[indArrayPartialApplyRawForward.Rest, Out]): Out = {
        val expScores = exp(scores)
        expScores / expScores.sum(1)
    }

    final class Model[Out](parent: Path,
                numberOfCoarseClasses: Int,
                numberOfFineClassesPerCoarseClass: Int,
                kernelSize: Int = 3, //F 卷积核的空间尺寸
                padding: Int = 1, //零填充数量
                stride: Int = 1 // 步长l
    )(implicit val implicitApplyRest: ImplicitApply.Aux[indArrayPartialApplyData.Rest, Out],
      asINDArrayWeight: Out <:< INDArrayWeight) {
        var version = {
            val versionFile = parent.resolve("version")
            if (Files.exists(versionFile)) {
                new String(Files.readAllBytes(versionFile), scala.io.Codec.UTF8.charSet).toInt
            } else {
                Files.createDirectories(parent)
                Files.write(versionFile, "0".getBytes("UTF-8"))
                0
            }
        }


        def loadOrInitialize[Out <: INDArrayWeight](
                relativePath: Path,
                initialValue: () => INDArray): WeightFile = {
            val versionedPath = parent.resolve(version.toString)
            val resolvedPath = versionedPath.resolve(relativePath)
            if (Files.exists(resolvedPath)) {
                logger.info(s"Loading weight from $resolvedPath...")
                val stream = new ObjectInputStream(new BufferedInputStream(Files.newInputStream(resolvedPath)))
                try {
                    val data = stream.readObject().asInstanceOf[INDArray]
                    val cache = stream.readObject().asInstanceOf[Option[INDArray]]
                    val weight: implicitApplyRest.Out = INDArrayWeight(data)
                    // TODO
                    // weight.cache = cache
                    WeightFile(weight, relativePath)
                } finally {
                    stream.close()
                }
            } else {
                logger.info(s"Initialize weight for $resolvedPath...")
                val weightFile = WeightFile(INDArrayWeight(initialValue()), relativePath)
                weightFile.backup(versionedPath)
                weightFile
            }
        }

        def convolutionThenRelu[Input, Weight, Bias, Out <: INDArrayLayer](
                input: Input,
                weight: Weight,
                bias: Bias)(
                 implicit inputDeepLearning: DeepLearning.Aux[Input, INDArray, INDArray],
                 weightDeepLearning: DeepLearning.Aux[Weight, INDArray, INDArray],
                 biasDeepLearning: DeepLearning.Aux[Bias, INDArray, INDArray],
                 layerImplicits: ImplicitApply.Aux[indArrayPartialApplyRawForward.Rest, Out]): Out = {
            max(conv2d(input, weight, bias, (kernelSize, kernelSize), (stride, stride), (padding, padding)), 0.0)
        }

    
        private def initialWeightAndBias(path: Path, outputDepth: Int, inputDepth: Int): (WeightFile, WeightFile) = {
            import org.nd4s.Implicits._
            val weight = loadOrInitialize(
                path.resolve("weight"),
                { () => 
                    Nd4j.randn(Array(outputDepth, inputDepth, kernelSize, kernelSize)) *
                    math.sqrt(2.0 / inputDepth / kernelSize / kernelSize)
                }
            )

            //When using RELUs, make sure biases are initialised with small *positive* values for example 0.1
            val bias = loadOrInitialize(
                path.resolve("bias"),                          
                () => Nd4j.ones(outputDepth) * 0.0001
            )

            (weight, bias)
        }

        def backup(): Unit = {
            val currentVersion = synchronized {
                version += 1
                version
            }
            val versionFile = parent.resolve("version")
            Files.write(versionFile, currentVersion.toString.getBytes("UTF-8"))
            val versionedPath = parent.resolve(version.toString)
            logger.info(s"Backing up model to $versionedPath")
            for ((weight, bias) <- coarseNetWeightAndBias) {
                weight.backup(versionedPath)
                bias.backup(versionedPath)
            }
            inputWeightAndBias._1.backup(versionedPath)
            inputWeightAndBias._2.backup(versionedPath)
            for ((weight, bias) <- coarseStage2WeightAndBias) {
                weight.backup(versionedPath)
                bias.backup(versionedPath)
            }
            coarseBias.backup(versionedPath)
            coarseWeight.backup(versionedPath)
            for (subnet <- fineNetWeightAndBiases) {
                for ((weight, bias) <- subnet) {
                    weight.backup(versionedPath)
                    bias.backup(versionedPath)
                }
            }
            for (bias <- fineBiases) {
                bias.backup(versionedPath)
            }
            for (weight <- fineWeights) {
                weight.backup(versionedPath)
            }
        }
        
        val inputWeightAndBias = initialWeightAndBias(Paths.get("coarse", "cnn" , "0"), NumberOfFilters, 3)
        
        val coarseNetWeightAndBias = {
            for (i <- 1 until 10) yield {
                initialWeightAndBias(Paths.get("coarse", "cnn", i.toString), NumberOfFilters, NumberOfFilters)
            }
        }

        val coarseStage2WeightAndBias = {
            for (i <- 0 until 5) yield {
                initialWeightAndBias(Paths.get("coarse", "cnn_stage2", i.toString), NumberOfFilters, NumberOfFilters)
            }
        }

        val coarseBias = loadOrInitialize(
            Paths.get("coarse", "fc", "bias"),
            () => Nd4j.zeros(numberOfCoarseClasses)
        )

        val coarseWeight = loadOrInitialize(
            Paths.get("coarse", "fc", "weight"),
            { () =>
                import org.nd4s.Implicits._
                Nd4j.randn(NumberOfFilters * 16 * 16, numberOfCoarseClasses) / math.sqrt(NumberOfFilters * 16 * 16)
            }
        )

        val fineNetWeightAndBiases = for (i <- 0 until numberOfCoarseClasses) yield {
            for (j <- 0 until 5) yield {
                initialWeightAndBias(Paths.get("fine", i.toString, "cnn", j.toString), NumberOfFilters, NumberOfFilters)
            }
        }

        val fineBiases = for (i <- 0 until numberOfCoarseClasses) yield {
            loadOrInitialize(
                Paths.get("fine", i.toString, "fc", "bias"),
                { () =>
                    Nd4j.zeros(numberOfFineClassesPerCoarseClass)
                }
            )
        }

        val fineWeights = {
            import org.nd4s.Implicits._
            for (i <- 0 until numberOfCoarseClasses) yield {
                loadOrInitialize(
                    Paths.get("fine", i.toString, "fc", "weight"),
                    { () =>
                        Nd4j.randn(NumberOfFilters * 16 * 16, numberOfFineClassesPerCoarseClass) / math.sqrt(NumberOfFilters * 16 * 16 / 2)
                    }
                )
            }
        }

    }


}

import java.io._

import java.net.URI

import java.nio.file._


defined trait BranchNet

In [10]:
interp.load("""
    val hyperparameters = Factory[BranchNet with Adagrad with FixedLearningRate with CNNs].newInstance(
        learningRate = 0.00001,
        eps = 1e-8
    )
""")

In [11]:
import hyperparameters.INDArrayWeight
import hyperparameters.DoubleLayer
import hyperparameters.INDArrayLayer
import hyperparameters.softmax
import hyperparameters.log
import hyperparameters.implicits._

import hyperparameters.INDArrayWeight

import hyperparameters.DoubleLayer

import hyperparameters.INDArrayLayer

import hyperparameters.softmax

import hyperparameters.log

import hyperparameters.implicits._

In [12]:
import $ivy.`com.thoughtworks.deeplearning.etl::cifar100:0.1`

import com.thoughtworks.deeplearning.etl.Cifar100

val cifar100 = Cifar100.load().blockingAwait

import $ivy.$                                                


import com.thoughtworks.deeplearning.etl.Cifar100


cifar100: com.thoughtworks.deeplearning.etl.Cifar100 = Cifar100(
  java.nio.DirectByteBufferR[pos=0 lim=153700000 cap=153700000],
  java.nio.DirectByteBufferR[pos=0 lim=30740000 cap=30740000]
)

In [13]:
object Models {
    val model = new hyperparameters.Model(
        Paths.get("backup"),
        Cifar100.NumberOfCoarseClasses,
        Cifar100.NumberOfFineClassesPerCoarseClass,
        3,
        1,
        1
    )
}

defined object Models

In [14]:
import Models.model._

import Models.model._

In [15]:
def coarseStage2Layers(features: INDArrayLayer): INDArrayLayer = {
    coarseStage2WeightAndBias.foldLeft(features) { (input, weightFile) =>
        convolutionThenRelu(input, weightFile._1.weight, weightFile._2.weight)
    }
}

defined function coarseStage2Layers

In [16]:
def coarseSubNet(images: INDArray): INDArrayLayer = {
    coarseNetWeightAndBias.foldLeft(convolutionThenRelu(images, inputWeightAndBias._1.weight, inputWeightAndBias._2.weight)) { (accumulator, weightAndBias) =>
        convolutionThenRelu(accumulator, weightAndBias._1.weight, weightAndBias._2.weight)
    }
}

defined function coarseSubNet

In [17]:
def fineSubNet(coarseClass: Int, features: INDArrayLayer): INDArrayLayer = {
    val output = fineNetWeightAndBiases(coarseClass).foldLeft(features) { (input, weightAndBias) =>
        convolutionThenRelu(input, weightAndBias._1.weight, weightAndBias._2.weight)
    }
    val poolSize = (2, 2)
    hyperparameters.maxPool(output, poolSize)
}

defined function fineSubNet

In [18]:
def fullyConnectedLayer(input: INDArrayLayer,
                        weight: INDArrayWeight,
                        bias: INDArrayWeight,
                        imageCount: Int,
                        pixelsOfPreLayer: Int): INDArrayLayer = {
    input.reshape(imageCount, pixelsOfPreLayer) dot weight + bias
}

defined function fullyConnectedLayer

In [19]:
def coarseMaxPoolThenFC(features: INDArrayLayer, imageCount: Int): INDArrayLayer = {
    // N * NumberOfFilters * 16 * 16
    val layer0 = hyperparameters.maxPool(features, (2, 2))
    // N * 20
    fullyConnectedLayer(layer0, coarseWeight.weight, coarseBias.weight, imageCount, NumberOfFilters * 16 * 16)
}

defined function coarseMaxPoolThenFC

In [20]:
def lossFunction(probabilities: INDArrayLayer, expectedLabel: INDArray): DoubleLayer = {
    -(hyperparameters.log(probabilities + 0.0001) * expectedLabel).mean
//     -(log(possibility * 0.9 + 0.1) * expectedLabel + (1.0 - expectedLabel) * log(1.0 - possibility * 0.9)).mean
}

defined function lossFunction

In [21]:
def fineFCThenSoftmax(features: INDArrayLayer,
                      coarseClass: Int,
                      imageCount: Int,
                      pixelsOfPreLayer: Int): INDArrayLayer = {
    val layer0 = fullyConnectedLayer(features, fineWeights(coarseClass).weight, fineBiases(coarseClass).weight, imageCount, pixelsOfPreLayer)
    softmax(layer0)
}

defined function fineFCThenSoftmax

In [22]:
type CoarseAccuracyRate = Double
type FineAccuracyRate = Double
type AccuracyRate = Double
type Loss = Double

defined type CoarseAccuracyRate
defined type FineAccuracyRate
defined type AccuracyRate
defined type Loss

In [23]:
def accuracyRate(coarsePossibility: INDArray,
                 finePossibility: INDArray,
                 coarseClass: Int,
                 fineClasses: INDArray,
                 imageCount: Int): (CoarseAccuracyRate, FineAccuracyRate, AccuracyRate) = {
    import org.nd4s.Implicits._

    // Some assertion to make sure the shape is expected.
    val Array(`imageCount`, Cifar100.NumberOfCoarseClasses) = coarsePossibility.shape()
    val Array(`imageCount`, Cifar100.NumberOfFineClassesPerCoarseClass) = finePossibility.shape()
    val Array(`imageCount`, Cifar100.NumberOfFineClassesPerCoarseClass) = fineClasses.shape()

    val isCoarseClassCorrect = Nd4j.argMax(coarsePossibility, 1) eq coarseClass
    val isFineClassCorrect = (Nd4j.argMax(finePossibility, 1) eq Nd4j.argMax(fineClasses, 1))
    val numberOfCoarseCorrectLabels = isCoarseClassCorrect.sumT
    val numberOfFineCorrectLabels = isFineClassCorrect.sumT
    val numberOfCorrectLabels = (isCoarseClassCorrect * isFineClassCorrect).sumT
    val coarseAccuracyRate = numberOfCoarseCorrectLabels / imageCount
    val fineAccuracyRate = numberOfFineCorrectLabels / imageCount
    val finalAccuracyRate = numberOfCorrectLabels / imageCount
  
    (coarseAccuracyRate, fineAccuracyRate, finalAccuracyRate)
}

defined function accuracyRate

In [24]:
def trainAndCalculateAccuracyRate(coarseClass: Int, trainData: Cifar100.Batch): Future[(Loss, CoarseAccuracyRate, FineAccuracyRate, AccuracyRate)] = {
  @monadic[Do]
  def doTrain(images: INDArray,
              fineClasses: INDArray,
              coarseClasses: INDArray,
              coarseClass: Int) = {
    val imageCount = images.shape()(0)
    // N * NumberOfFilters * 32 * 32
    val coarseCNNOutput = coarseSubNet(images)
    
    val coarseStage2Output = coarseStage2Layers(coarseCNNOutput)
    
    // N * 20
    val coarseSoftMaxOutput = softmax(coarseMaxPoolThenFC(coarseStage2Output, imageCount))
    val coarseLoss = lossFunction(coarseSoftMaxOutput, coarseClasses)

    val fineOutput = fineSubNet(coarseClass, coarseCNNOutput)
    val fineSoftMaxOutput =
      fineFCThenSoftmax(fineOutput, coarseClass, imageCount, NumberOfFilters * 16 * 16)
    val fineLoss = lossFunction(fineSoftMaxOutput, fineClasses)
    val lossLayer = coarseLoss + fineLoss

    val lossTape: Tape[Double, Double] = lossLayer.forward.each

    val backwardFuture = lossTape.backward(Do.now(1.0))
    Do.garbageCollected(backwardFuture).each

    val loss: Double = lossTape.data

    val coarsePossibility = coarseSoftMaxOutput.forward.each.data
    val finePossibility = fineSoftMaxOutput.forward.each.data

    val (coarseAccuracyRate, fineAccuracyRate, finalAccuracyRate) = accuracyRate(coarsePossibility, finePossibility, coarseClass, fineClasses, imageCount)

    (loss, coarseAccuracyRate, fineAccuracyRate, finalAccuracyRate)
  }

  doTrain(trainData.pixels, trainData.localFineClasses, trainData.coarseClasses, coarseClass).run
}

defined function trainAndCalculateAccuracyRate

In [25]:
def predict(images: INDArray): Future[(Int, Int)] = {

  @monadic[Do]
  def doPredict(images: INDArray, imageCount: Int): Do[(Int, Int)] = {
    import org.nd4s.Implicits._
    val coarseCNNOutput = coarseSubNet(images)
    val coarseFCOutput = coarseMaxPoolThenFC(coarseCNNOutput, imageCount)
    val coarseClass: Int = Nd4j.argMax(coarseFCOutput.forward.each.data, 1).get(0).toInt

    val fineOutput = fineSubNet(coarseClass, coarseCNNOutput)
    val fineSoftMaxOutput = fineFCThenSoftmax(fineOutput, coarseClass, imageCount, NumberOfFilters * 16 * 16)
    val fineClass: Int = Nd4j.argMax(fineSoftMaxOutput.forward.each.data, 1).get(0).toInt
    (coarseClass, fineClass)
  }

  val imageCount = images.shape()(0)

  doPredict(images, imageCount).run
}


defined function predict

In [26]:
import scalaz.Semigroup
import com.thoughtworks.deeplearning.plugins.INDArrayLayers.MultipleException
implicit val throwableSemigroup = new Semigroup[Throwable] {
  override def append(f1: Throwable, f2: => Throwable): Throwable =
    f1 match {
      case MultipleException(exceptionSet1) =>
        f2 match {
          case MultipleException(exceptionSet2) => MultipleException(exceptionSet1 ++ exceptionSet2)
          case _: Throwable                     => MultipleException(exceptionSet1 + f2)
        }
      case _: Throwable =>
        f2 match {
          case MultipleException(exceptionSet2) => MultipleException(exceptionSet2 + f1)
          case _: Throwable                     => MultipleException(Set(f1, f2))
        }
    }
}

import scalaz.Semigroup

import com.thoughtworks.deeplearning.plugins.INDArrayLayers.MultipleException

throwableSemigroup: AnyRef with scalaz.Semigroup[Throwable] = $sess.cmd25Wrapper$Helper$$anon$1@2c1ba05f

In [27]:
@volatile
var isShuttingDown: Boolean = false

def shutdown(): Unit = {
    isShuttingDown = true
}

isShuttingDown: Boolean = false
defined function shutdown

In [28]:
type CoarseClass = Int
type IterationId = Int
final class Epoch(epochId: Int, batchSize: Int, maxSamplesPerCoarseClass: Int) {
    val batches: Iterator[(CoarseClass, Cifar100.Batch)] = cifar100.epochByCoarseClass(batchSize, maxSamplesPerCoarseClass)
    var nextIterationId = 0
    def nextBatch(): Option[((CoarseClass, Cifar100.Batch), IterationId)] = {
        synchronized {
            if (batches.hasNext) {
                val currentIterationId = nextIterationId
                if (currentIterationId % 100 == 0) {
                    backup()
                }
                nextIterationId += 1
                val result = ((batches.next()), currentIterationId)
                Some(result)
            } else {
                None
            }
        }
    }

    def run(): Future[Unit] = {
        nextBatch() match {
            case Some(((coarseClass, batch), i)) if !isShuttingDown => 
                trainAndCalculateAccuracyRate(coarseClass, batch).flatMap {
                    case (loss, coarseAccuracyRate, fineAccuracyRate, accuracyRate) =>
                        hyperparameters.logger.info(f"epoch=$epochId%03d\titeration=$i%05d\tbatchSize=$batchSize\tcoarseClass=$coarseClass%02d\tloss=$loss%.7f\tcoarseAccuracyRate=$coarseAccuracyRate%.3f\tfineAccuracyRate=$fineAccuracyRate%.3f\taccuracyRate=$accuracyRate%.3f")
                        run()
                }   
            case _ =>
                Future.now(())                     
        }
    }
}

defined type CoarseClass
defined type IterationId
defined class Epoch

In [29]:
def startEpoch(batchSize: Int, numberOfEpoches: Int, numberOfConcurrenctTrainings: Int, maxSamplesPerCoarseClass: Int): Unit = {
    isShuttingDown = false
    import scalaz.Monoid
    import scalaz.Tags.Parallel

    @monadic[Future]
    def trainTask: Future[Unit] = {
        for (epochId <- 1 |=> numberOfEpoches) {
            hyperparameters.logger.info(s"Epoch $epochId")
            val epoch = new Epoch(epochId, batchSize, maxSamplesPerCoarseClass)
            
            val fork = (1 |=> numberOfConcurrenctTrainings).foldMap { futureId: Int =>
                Parallel(epoch.run())
            }(Monoid.liftMonoid[ParallelFuture, Unit](futureParallelApplicative, scalaz.std.anyVal.unitInstance))
            Parallel.unwrap(fork).each
        }
    }

    trainTask.onComplete {
        case util.Success(s) =>
        case util.Failure(e) =>
            hyperparameters.logger.log(java.util.logging.Level.SEVERE, "Exception thrown during training", e)
    }
}

defined function startEpoch

In [30]:
sys.props("org.bytedeco.javacpp.maxbytes") = "28g"

In [32]:
startEpoch(batchSize = 4, numberOfEpoches = 500, numberOfConcurrenctTrainings = 4, maxSamplesPerCoarseClass = 28)

In [32]:
shutdown()

In [34]:
backup()

In [29]:




// @monadic[Future]
// def trainTask: Future[Unit] = {
//   val lossStream: Stream[TrainResult] =
//     (for (_ <- (0 until 200).toStream) yield {
//       val randomIndex = Utils.getRandomIndex(NumberOfTrainSamples)
//       (for (times <- (0 until NumberOfTrainSamples / batchSize).toStream) yield {
//         val slicedIndexArray = Utils.sliceIndexArray(randomIndex, times, MiniBatchSize)
//         val trainData = ReadCIFARToNDArray.processSGDTrainData(slicedIndexArray)
//         for (index <- trainData.indices.toStream) yield {
//           val currentInput = trainData(index)
//           val futureTrainResult: Future[TrainResult] = train(currentInput)
//           val trainResult = futureTrainResult.each
//           println("loss:" + trainResult.loss)
//           println("coarseAcc:" + trainResult.coarseAcc)
//           println("fineAcc:" + trainResult.fineAcc)
//           trainResult
//         }
//       }).flatten
//     }).flatten
//   trainResultSeq = IndexedSeq.concat(lossStream)
// }

// var lossSeq: IndexedSeq[Double] = IndexedSeq.empty

// @monadic[Future]
// val trainTask0: Future[Unit] = {
//   val lossStream: Stream[Double] =
//     (for (_ <- (0 until 500).toStream) yield {
//       System.gc()
//       val randomIndex = Utils.getRandomIndex(NumberOfTrainSamples)
//       (for (times <- (0 until NumberOfTrainSamples / batchSize).toStream) yield {
//         val slicedIndexArray =
//           Utils.sliceIndexArray(randomIndex, times, MiniBatchSize)
//         val trainData =
//           ReadCIFARToNDArray.processSGDTrainData(slicedIndexArray)
//         for (index <- trainData.indices.toStream) yield {
//           val currentInput = trainData(index)
//           val futureTrainResult: Future[Double] = train0(currentInput)
//           val loss = futureTrainResult.each
//           println("loss:" + loss)

//           if (loss.isNaN) {
//             sys.exit(-11111111)
//           }

//           loss
//         }
//       }).flatten
//     }).flatten
//   lossSeq = IndexedSeq.concat(lossStream)
// }

// val startTime = LocalTime.now()
// //Await.result(trainTask.toScalaFuture, Duration.Inf)
// Await.result(trainTask0.toScalaFuture, Duration.Inf)
// val endTime = LocalTime.now()

// //  private val resultTuple
// //    : (IndexedSeq[Double], IndexedSeq[Double], IndexedSeq[Double]) =
// //    trainResultSeq.unzip3(result =>
// //      (result.loss, result.coarseAcc, result.fineAcc))
// //
// //  Seq(
// //    Scatter(trainResultSeq.indices, resultTuple._1, name = "loss"),
// //    Scatter(trainResultSeq.indices, resultTuple._2, name = "coarseAcc"),
// //    Scatter(trainResultSeq.indices, resultTuple._3, name = "fineAcc")
// //  ).plot(title = "loss & acc by time")

// def calculatePredictAcc(): (Double, Double) = {

//   def calculatePredictResult: IndexedSeq[(Int, Int)] = {
//     for (testItem <- testData)
//       yield {
//         val futurePredictResult: Future[PredictResult] = predict(testItem.imageData)
//         val PredictResult(predictCoarse, predictFine) =
//           Await.result(futurePredictResult.toScalaFuture, Duration.Inf)
//         if (predictCoarse == testItem.coarse) {
//           (1, Utils.isSameThenOne(predictFine, testItem.fine))
//         } else {
//           (0, 0)
//         }
//       }
//   }

//   val predictResult: IndexedSeq[(Int, Int)] = calculatePredictResult
//   val resultTuple: (IndexedSeq[Int], IndexedSeq[Int]) = predictResult.unzip
//   val total = resultTuple._1.size
//   (resultTuple._1.sum / total, resultTuple._2.sum / total)
// }

// val (coarseAcc, fineAcc) = calculatePredictAcc()

// println(s"The coarse accuracy is $coarseAcc ,The fine accuracy is $fineAcc , start at $startTime , end at $endTime")

// Seq(
//   Scatter(lossSeq.indices, lossSeq, name = "loss")
// ).plot(title = s"loss by time - learningRate : $learningRate, start at $startTime , end at $endTime")